<a href="https://colab.research.google.com/github/emumily/fetch-assignment/blob/br1/Fetch_Assignment_EmilyTang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Time 2/18 Tuesday 6PM

# Exercise:
## First: explore the data
Review the unstructured csv files and answer the following questions with code that supports your conclusions:

**Are there any data quality issues present?\
Are there any fields that are challenging to understand?**

We recommend using SQL or python and data visualization to examine the data.

# Second: **provide SQL queries**
Answer three of the following questions with at least one question coming from the closed-ended and one from the open-ended question set. *Each question should be answered using one query.*

#### Closed-ended questions:
**What are the top 5 brands by receipts scanned among users 21 and over?\
What are the top 5 brands by sales among users that have had their account for at least six months?\
What is the percentage of sales in the Health & Wellness category by generation?**
#### Open-ended questions: make assumptions and clearly state them.
**Who are Fetch’s power users?\
Which is the leading brand in the Dips & Salsa category?\
At what percent has Fetch grown year over year?**

# Third: communicate with stakeholders
**Construct an email or slack message that is understandable to a product or business leader who is not familiar with your day-to-day work. Summarize the results of your investigation.** \
Include:
1. Key data quality issues and outstanding questions about the data\
2. One interesting trend in the data \
3. Use a finding from part 2 or come up with a new insight \
4.
Request for action: explain what additional help, info, etc. you need to make sense of the data and resolve any outstanding issues

In [ ]:
First: explore the data
Review the unstructured csv files and answer the following questions with code that supports your conclusions:

Are there any data quality issues present?
Are there any fields that are challenging to understand?

In [1]:
# since I'm using Google Colab, I need to upload the CSV files to my drive (NOT on Github)
# This function pulls up the Finder for me to select the CSV files, and then uploads them to the "uploaded" object.
from google.colab import files
uploaded = files.upload()

Saving PRODUCTS_TAKEHOME.csv to PRODUCTS_TAKEHOME.csv
Saving TRANSACTION_TAKEHOME.csv to TRANSACTION_TAKEHOME.csv
Saving USER_TAKEHOME.csv to USER_TAKEHOME.csv


In [44]:
import pandas as pd
import io

# Import products, transactions, and users data files (csv)
products = pd.read_csv(io.BytesIO(uploaded['PRODUCTS_TAKEHOME.csv']))
transactions = pd.read_csv(io.BytesIO(uploaded['TRANSACTION_TAKEHOME.csv']))
users = pd.read_csv(io.BytesIO(uploaded['USER_TAKEHOME.csv']))

In [10]:
# How many rows x columns for each dataset?
products.shape , transactions.shape, users.shape

((845552, 7), (50000, 8), (100000, 6))

#### Products Table Exploration

Barcode is the only numeric value (which we should treat as a character)
Lots of NaN (missing values)

In [9]:
products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


In [57]:
import numpy as np
# Missing values NaN, empty cells, possibly null - account for these by making them all NaN
products = products.replace(r'^\s*$', np.nan, regex=True)

In [58]:
# With each breakdown of the category, there are more missing values, which makes sense as products might not fit into specific subcategories.
# Manufacturer and Brand are quite high in missing percentage, around ~27%.
missing_counts = products.isna().sum()
missing_percent = 100 * (missing_counts / len(products))

missing_summary = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_percent})
print(missing_summary)

              Missing Count  Missing %
CATEGORY_1              111   0.013128
CATEGORY_2             1424   0.168411
CATEGORY_3            60566   7.162895
CATEGORY_4           778093  92.021898
MANUFACTURER         226474  26.784160
BRAND                226472  26.783923
BARCODE                4025   0.476020


In [52]:
transactions.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


In [35]:
# Transactions has some blank, some not of consistent type
# (eg "zero", a string and 1.00, a float)
# having difficulty understanding why FINAL_QUANTITY could be zero, but FINAL_SALE is >0
transactions['FINAL_QUANTITY'].dtype # object

dtype('O')

In [54]:
transactions['FINAL_QUANTITY'].value_counts()

,count
FINAL_QUANTITY,
1.00,35698
zero,12500
2.00,1285
3.00,184
4.00,139
...,...
6.22,1
1.22,1
1.23,1


In [59]:
# Missing values NaN, empty cells, possibly null - account for these by making them all NaN
# transactions.replace({"": np.nan, "NULL": np.nan, "NA": np.nan}, inplace=True) originally did this, but it wasn't affecting the object (FINAL_SALE)
transactions = transactions.replace(r'^\s*$', np.nan, regex=True)

In [60]:
missing_counts = transactions.isna().sum()
missing_percent = 100 * (missing_counts / len(transactions))

missing_summary = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_percent})
print(missing_summary)

                Missing Count  Missing %
RECEIPT_ID                  0      0.000
PURCHASE_DATE               0      0.000
SCAN_DATE                   0      0.000
STORE_NAME                  0      0.000
USER_ID                     0      0.000
BARCODE                  5762     11.524
FINAL_QUANTITY              0      0.000
FINAL_SALE              12500     25.000


In [42]:
missing_counts = transactions.isna().sum()
missing_percent = 100 * (missing_counts / len(transactions))

missing_summary = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_percent})
print(missing_summary)

                Missing Count  Missing %
RECEIPT_ID                  0      0.000
PURCHASE_DATE               0      0.000
SCAN_DATE                   0      0.000
STORE_NAME                  0      0.000
USER_ID                     0      0.000
BARCODE                  5762     11.524
FINAL_QUANTITY              0      0.000
FINAL_SALE                  0      0.000


In [61]:
# I also want to convert "zero" to 0.0 in the "FINAL_QUANTITY" variable
transactions['FINAL_QUANTITY'] = transactions['FINAL_QUANTITY'].replace('zero', 0.0).astype(float)

In [62]:
# double checking - looks good for now?
transactions.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.0,NaN
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,0.0,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.0,NaN
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,0.0,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.0,NaN


In [8]:
users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female
